In [1]:
import os
import requests
from datetime import datetime
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io

In [2]:
# %%
# Charger le fichier .env
load_dotenv()
# Récupération des variables
AZURE_CONNECTION_STRING = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
AZURE_ACCOUNT_NAME = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')
# Vérification
if not AZURE_CONNECTION_STRING:
    raise ValueError("❌ La chaîne de connexion Azure n'est pas définie dans le fichier .env")
print("✅ Variables d'environnement chargées avec succès")
print(f"📦 Compte de stockage : {AZURE_ACCOUNT_NAME}")


✅ Variables d'environnement chargées avec succès
📦 Compte de stockage : datalakequaliteeauimpe


In [3]:
# %%
# Dictionnaire des URLs des fichiers ZIP par année
DATA_URLS = {
    2025: "https://www.data.gouv.fr/api/1/datasets/r/6994a9f1-3f4b-4e15-a4dc-0e358a6aac13",
    2024: "https://www.data.gouv.fr/api/1/datasets/r/c0350599-a041-4724-9942-ad4c2ba9a7b3",
    2023: "https://www.data.gouv.fr/api/1/datasets/r/96452cf0-329a-4908-8adb-8f061adcca4c",
    2022: "https://www.data.gouv.fr/api/1/datasets/r/77d3151a-739e-4aab-8c34-7a15d7fea55d",
    2021: "https://www.data.gouv.fr/api/1/datasets/r/3c5ebbd9-f6b5-4837-a194-12bfeda7f38e"
}
# Conteneurs Azure
CONTAINER_RAW = "raw"
CONTAINER_BRONZE = "bronze"  # Pour usage futur
print(f"📊 Nombre de fichiers à récupérer : {len(DATA_URLS)}")
# %% [markdown]
# ## 5. Connexion au Blob Storage Azure
# %%
try:
    # Créer le client de service Blob
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
    
    # Vérifier la connexion en listant les conteneurs
    containers = blob_service_client.list_containers()
    container_names = [container.name for container in containers]
    
    print("✅ Connexion au Blob Storage réussie")
    print(f"📂 Conteneurs disponibles : {container_names}")
    
    # Vérifier que le conteneur 'raw' existe
    if CONTAINER_RAW not in container_names:
        print(f"⚠️  Le conteneur '{CONTAINER_RAW}' n'existe pas. Création en cours...")
        blob_service_client.create_container(CONTAINER_RAW)
        print(f"✅ Conteneur '{CONTAINER_RAW}' créé avec succès")
    
except Exception as e:
    print(f"❌ Erreur de connexion au Blob Storage : {str(e)}")
    raise
# %% [markdown]
# ## 6. Fonction de téléchargement et upload vers Azure
# %%
def download_and_upload_to_blob(year, url, container_name):
    """
    Télécharge un fichier ZIP depuis une URL et l'upload vers Azure Blob Storage
    
    Args:
        year (int): Année des données
        url (str): URL du fichier ZIP
        container_name (str): Nom du conteneur Azure
    
    Returns:
        bool: True si succès, False sinon
    """
    try:
        print(f"\n{'='*60}")
        print(f"📥 Téléchargement des données {year}...")
        
        # Télécharger le fichier
        response = requests.get(url, stream=True, timeout=300)
        response.raise_for_status()
        
        # Taille du fichier
        file_size = len(response.content)
        print(f"📦 Taille du fichier : {file_size / (1024*1024):.2f} MB")
        
        # Nom du blob avec timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        blob_name = f"qualite_eau_{year}_{timestamp}.zip"
        
        # Créer le client blob
        blob_client = blob_service_client.get_blob_client(
            container=container_name,
            blob=blob_name
        )
        
        # Upload vers Azure Blob Storage
        print(f"☁️  Upload vers Azure Blob Storage...")
        blob_client.upload_blob(
            response.content,
            overwrite=True,
            metadata={
                "year": str(year),
                "source": "data.gouv.fr",
                "upload_date": timestamp,
                "original_url": url
            }
        )
        
        print(f"✅ Fichier uploadé avec succès : {blob_name}")
        print(f"📍 Chemin : {container_name}/{blob_name}")
        
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors du téléchargement : {str(e)}")
        return False
    except Exception as e:
        print(f"❌ Erreur lors de l'upload : {str(e)}")
        return False


📊 Nombre de fichiers à récupérer : 5
✅ Connexion au Blob Storage réussie
📂 Conteneurs disponibles : ['bronze', 'raw']


In [4]:
# %% [markdown]
# ## 7. Exécution du pipeline d'ingestion
# %%
print("\n" + "="*60)
print("🚀 DÉMARRAGE DU PIPELINE D'INGESTION")
print("="*60)
# Statistiques
success_count = 0
failed_count = 0
failed_years = []
# Traiter chaque fichier
for year, url in sorted(DATA_URLS.items(), reverse=True):
    success = download_and_upload_to_blob(year, url, CONTAINER_RAW)
    
    if success:
        success_count += 1
    else:
        failed_count += 1
        failed_years.append(year)
# %% [markdown]
# ## 8. Rapport final
# %%
print("\n" + "="*60)
print("📊 RAPPORT FINAL")
print("="*60)
print(f"✅ Fichiers uploadés avec succès : {success_count}/{len(DATA_URLS)}")
print(f"❌ Fichiers en échec : {failed_count}/{len(DATA_URLS)}")
if failed_years:
    print(f"⚠️  Années en échec : {failed_years}")
else:
    print("🎉 Tous les fichiers ont été traités avec succès!")
print("="*60)
# %% [markdown]
# ## 9. Vérification des fichiers dans le conteneur RAW
# %%
print("\n📂 CONTENU DU CONTENEUR RAW :")
print("="*60)
container_client = blob_service_client.get_container_client(CONTAINER_RAW)
blob_list = container_client.list_blobs()
for idx, blob in enumerate(blob_list, 1):
    print(f"\n{idx}. {blob.name}")
    print(f"   📏 Taille : {blob.size / (1024*1024):.2f} MB")
    print(f"   📅 Dernière modification : {blob.last_modified}")
    if blob.metadata:
        print(f"   📋 Metadata : {blob.metadata}")
print("\n" + "="*60)
print("✨ Pipeline terminé!")


🚀 DÉMARRAGE DU PIPELINE D'INGESTION

📥 Téléchargement des données 2025...
📦 Taille du fichier : 175.37 MB
☁️  Upload vers Azure Blob Storage...
✅ Fichier uploadé avec succès : qualite_eau_2025_20251027_131455.zip
📍 Chemin : raw/qualite_eau_2025_20251027_131455.zip

📥 Téléchargement des données 2024...
📦 Taille du fichier : 274.63 MB
☁️  Upload vers Azure Blob Storage...
✅ Fichier uploadé avec succès : qualite_eau_2024_20251027_131545.zip
📍 Chemin : raw/qualite_eau_2024_20251027_131545.zip

📥 Téléchargement des données 2023...
📦 Taille du fichier : 278.48 MB
☁️  Upload vers Azure Blob Storage...
✅ Fichier uploadé avec succès : qualite_eau_2023_20251027_131644.zip
📍 Chemin : raw/qualite_eau_2023_20251027_131644.zip

📥 Téléchargement des données 2022...
📦 Taille du fichier : 293.85 MB
☁️  Upload vers Azure Blob Storage...
✅ Fichier uploadé avec succès : qualite_eau_2022_20251027_131745.zip
📍 Chemin : raw/qualite_eau_2022_20251027_131745.zip

📥 Téléchargement des données 2021...
📦 Taille 